In [1]:
import gym
from collections import namedtuple
import numpy as np

import tensorflow as tf
import tensorflow.keras.layers as layers

In [2]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [3]:
class Net(tf.keras.Model):
    def get_config(self):
        pass
    def __init__(self, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = tf.keras.Sequential([
            layers.Dense(hidden_size),
            layers.ReLU(),
            layers.Dense(n_actions)
        ])

    def call(self, inputs, training=None, mask=None):
        return self.net(inputs)

In [4]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple("EpisodeStep", field_names=['observation', 'action'])

In [5]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    softmax = layers.Softmax(axis=1)
    while True:
        obs_v = tf.convert_to_tensor([obs])
        act_probs_v = softmax(net(obs_v))
        act_probs = act_probs_v.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs



In [6]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))

    train_obs_v = tf.convert_to_tensor(train_obs)
    train_act_v = tf.convert_to_tensor(train_act, dtype=tf.int64)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [8]:
env = gym.make("CartPole-v0")
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(HIDDEN_SIZE, n_actions)
objective = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

log_dir = "logs/cartpole_cross_entropy"
writer = tf.summary.create_file_writer(logdir=log_dir)

for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
    with tf.GradientTape() as g:
        action_scores_v = net(obs_v)
        loss_v = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(acts_v, action_scores_v))

    gradients = g.gradient(loss_v, sources=net.trainable_variables)
    optimizer.apply_gradients(zip(gradients, net.trainable_variables))

    print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
        iter_no, loss_v.numpy(), reward_m, reward_b))
    with writer.as_default():
        tf.summary.scalar("loss", loss_v, iter_no)
        tf.summary.scalar("reward_bound", reward_b, iter_no)
        tf.summary.scalar("reward_mean", reward_m, iter_no)

    if reward_m > 199:
        print("Solved!")
        break



0: loss=0.693, reward_mean=23.2, reward_bound=25.0
1: loss=0.679, reward_mean=19.4, reward_bound=19.0
2: loss=0.673, reward_mean=27.9, reward_bound=31.0
3: loss=0.666, reward_mean=27.0, reward_bound=31.5
4: loss=0.660, reward_mean=29.4, reward_bound=33.0
5: loss=0.623, reward_mean=36.5, reward_bound=49.0
6: loss=0.620, reward_mean=49.8, reward_bound=59.0
7: loss=0.587, reward_mean=54.2, reward_bound=62.0
8: loss=0.595, reward_mean=57.9, reward_bound=65.5
9: loss=0.539, reward_mean=60.9, reward_bound=67.5
10: loss=0.554, reward_mean=65.2, reward_bound=61.5
11: loss=0.531, reward_mean=58.1, reward_bound=62.0
12: loss=0.513, reward_mean=60.7, reward_bound=60.5
13: loss=0.480, reward_mean=56.5, reward_bound=62.5
14: loss=0.495, reward_mean=73.8, reward_bound=75.5
15: loss=0.497, reward_mean=66.5, reward_bound=67.0
16: loss=0.459, reward_mean=90.6, reward_bound=101.5
17: loss=0.467, reward_mean=102.8, reward_bound=118.0
18: loss=0.454, reward_mean=115.7, reward_bound=134.0
19: loss=0.425, r